In [1]:
import pandas as pd

In [ ]:
df2007 = pd.read_csv("rendimento_escolar/Dados Tratados/dadosTratados2007.csv")
df2007

In [ ]:
matriculas_df = pd.read_csv("alunos_matriculados/matriculas_detalhadas_2007.csv")
matriculas_df

#### filtragem de total Rural E Urbano

In [11]:
# Lista das colunas que você quer manter (adicione as suas)
colunas_selecionadas = [
    'Ano', 
    'Região', 
    'UF', 
    'Código do Município', 
    'Nome do Município',
    'Localização',
    'Rede',
    'Total_Abandono_EF2007', 
    'Total_Abandono_EM2007',
    
]

# Filtro + seleção de colunas em uma única operação
dados_tratados_df = df2007[
   ( (df2007['Localização'] == 'Rural') | (df2007['Localização'] == 'Urbana'))
    & (df2007['Rede'] == 'Total')
][colunas_selecionadas].copy()

# Resetar índice (opcional)
dados_tratados_df = dados_tratados_df.reset_index(drop=True)

In [12]:
dados_tratados_df

,Ano,Região,UF,Código do Município,Nome do Município,Localização,Rede,Total_Abandono_EF2007,Total_Abandono_EM2007
0,2007,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Total,6.2,--
1,2007,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Urbana,Total,5.5,18.2
2,2007,Norte,RO,1100023,ARIQUEMES,Rural,Total,6.4,--
3,2007,Norte,RO,1100023,ARIQUEMES,Urbana,Total,3.1,11.3
4,2007,Norte,RO,1100031,CABIXI,Rural,Total,0,--
...,...,...,...,...,...,...,...,...,...
5251,2007,Centro_Oeste,GO,5222203,VILA BOA,Urbana,Total,1.4,13.5
5252,2007,Centro_Oeste,GO,5222302,VILA PROPICIO,Rural,Total,5.8,13.3
5253,2007,Centro_Oeste,GO,5222302,VILA PROPICIO,Urbana,Total,7.1,14.8
5254,2007,Centro_Oeste,DF,5300108,BRASILIA,Rural,Total,2.1,5.3


#### tornas as linhas RURAL e URBANA em colunas 
#### ainda não esta sendo usada

In [5]:

# Supondo que seu dataframe se chame df_filtrado
df_pivoted = df_filtrado.pivot(
    index=['Ano', 'Região', 'UF', 'Código do Município', 'Nome do Município', 'Rede'],
    columns='Localização',
    values=['Total_Abandono_EF2007', 'Total_Abandono_EM2007']
)

# Resetar o índice para transformar os níveis do multi-índice em colunas normais
df_pivoted = df_pivoted.reset_index()

# Renomear as colunas para ficarem mais claras
df_pivoted.columns = [
    'Ano', 'Região', 'UF', 'Código do Município', 'Nome do Município', 'Rede',
    'Total_Abandono_EF2007_Rural', 'Total_Abandono_EF2007_Urbana',
    'Total_Abandono_EM2007_Rural', 'Total_Abandono_EM2007_Urbana'
]

# Reordenar as colunas se necessário
df_pivoted = df_pivoted[[
    'Ano', 'Região', 'UF', 'Código do Município', 'Nome do Município', 'Rede',
    'Total_Abandono_EF2007_Rural', 'Total_Abandono_EF2007_Urbana',
    'Total_Abandono_EM2007_Rural', 'Total_Abandono_EM2007_Urbana'
]]

In [6]:
df_pivoted

,Ano,Região,UF,Código do Município,Nome do Município,Rede,Total_Abandono_EF2007_Rural,Total_Abandono_EF2007_Urbana,Total_Abandono_EM2007_Rural,Total_Abandono_EM2007_Urbana
0,2007,Centro_Oeste,DF,5300108,BRASILIA,Total,2.1,1.8,5.3,8
1,2007,Centro_Oeste,GO,5200050,ABADIA DE GOIAS,Total,2.4,8.9,--,25.6
2,2007,Centro_Oeste,GO,5200100,ABADIANIA,Total,3.1,5.7,0,18.4
3,2007,Centro_Oeste,GO,5200134,ACREUNA,Total,NaN,6.4,NaN,14.5
4,2007,Centro_Oeste,GO,5200159,ADELANDIA,Total,NaN,1.9,NaN,14
...,...,...,...,...,...,...,...,...,...,...
2703,2007,Norte,TO,1721208,TOCANTINOPOLIS,Total,17.8,1.6,25.6,18.6
2704,2007,Norte,TO,1721257,TUPIRAMA,Total,0,3.4,--,6.4
2705,2007,Norte,TO,1721307,TUPIRATINS,Total,3.8,2.1,--,9.2
2706,2007,Norte,TO,1722081,WANDERLANDIA,Total,5.3,0.9,--,11.3


#### mescla os dois dataframes para incluir as colunas Total_Rural e Total_Urbano da planilha matriculas_detalhadas

In [34]:
# Limpeza dos nomes das colunas para evitar problemas de formatação
matriculas_df.columns = matriculas_df.columns.str.strip()
dados_tratados_df.columns = dados_tratados_df.columns.str.strip()

# Converter o código do município em ambas as tabelas para garantir que eles são compatíveis
matriculas_df['CO_MUNICIPIO'] = matriculas_df['CO_MUNICIPIO'].astype(str)
dados_tratados_df['Código do Município'] = dados_tratados_df['Código do Município'].astype(str)

# Merge das tabelas com base no código do município
merged_df = pd.merge(matriculas_df, dados_tratados_df, left_on='CO_MUNICIPIO', right_on='Código do Município')

# Verificar se a junção foi bem-sucedida e visualizar as primeiras linhas
merged_df[['Ano','CO_MUNICIPIO','Nome do Município','Localização', 'Total_Urbana','Total_Rural', 'Total_Abandono_EF2007', 'Total_Abandono_EM2007']]


,Ano,CO_MUNICIPIO,Nome do Município,Localização,Total_Urbana,Total_Rural,Total_Abandono_EF2007,Total_Abandono_EM2007
0,2007,1100015,ALTA FLORESTA D'OESTE,Rural,7687.0,2596.0,6.2,--
1,2007,1100015,ALTA FLORESTA D'OESTE,Urbana,7687.0,2596.0,5.5,18.2
2,2007,1100023,ARIQUEMES,Rural,37912.0,3607.0,6.4,--
3,2007,1100023,ARIQUEMES,Urbana,37912.0,3607.0,3.1,11.3
4,2007,1100031,CABIXI,Rural,2970.0,51.0,0,--
...,...,...,...,...,...,...,...,...
5251,2007,5222203,VILA BOA,Urbana,1974.0,66.0,1.4,13.5
5252,2007,5222302,VILA PROPICIO,Rural,959.0,856.0,5.8,13.3
5253,2007,5222302,VILA PROPICIO,Urbana,959.0,856.0,7.1,14.8
5254,2007,5300108,BRASILIA,Rural,850528.0,29842.0,2.1,5.3


#### realiza a multiplicação 

In [36]:

# Verificar os tipos das colunas antes de realizar a multiplicação
print(merged_df[['Total_Urbana','Total_Rural', 'Total_Abandono_EF2007', 'Total_Abandono_EM2007']].dtypes)

# Converter explicitamente para float, caso não estejam nesse formato
merged_df['Total_Urbana'] = pd.to_numeric(merged_df['Total_Urbana'], errors='coerce')
merged_df['Total_Rural'] = pd.to_numeric(merged_df['Total_Rural'], errors='coerce')
merged_df['Total_Abandono_EF2007'] = pd.to_numeric(merged_df['Total_Abandono_EF2007'], errors='coerce')
merged_df['Total_Abandono_EM2007'] = pd.to_numeric(merged_df['Total_Abandono_EM2007'], errors='coerce')

# Substituir NaN por 0 para evitar multiplicação com valores ausentes
merged_df['Total_Abandono_EM2007'] = merged_df['Total_Abandono_EM2007'].fillna(0)
merged_df['Total_Abandono_EF2007'] = merged_df['Total_Abandono_EF2007'].fillna(0)

# Realizar a multiplicação
merged_df['Resultado_Multiplicacao_EF2007'] = (merged_df['Total_Urbana'] *
                                        merged_df['Total_Abandono_EF2007'])

merged_df['Resultado_Multiplicacao_EM2007'] = (merged_df['Total_Urbana'] *
                                        merged_df['Total_Abandono_EM2007'])

# Exibir as primeiras linhas com o resultado da multiplicação
print(merged_df[['CO_MUNICIPIO', 'Nome do Município', 'Total_Urbana','Total_Rural', 'Total_Abandono_EF2007', 'Total_Abandono_EM2007', 'Resultado_Multiplicacao_EF2007','Resultado_Multiplicacao_EM2007']].head())


Total_Urbana             float64
Total_Rural              float64
Total_Abandono_EF2007    float64
Total_Abandono_EM2007    float64
dtype: object
  CO_MUNICIPIO      Nome do Município  Total_Urbana  Total_Rural  \
0      1100015  ALTA FLORESTA D'OESTE        7687.0       2596.0   
1      1100015  ALTA FLORESTA D'OESTE        7687.0       2596.0   
2      1100023              ARIQUEMES       37912.0       3607.0   
3      1100023              ARIQUEMES       37912.0       3607.0   
4      1100031                 CABIXI        2970.0         51.0   

   Total_Abandono_EF2007  Total_Abandono_EM2007  \
0                    6.2                    0.0   
1                    5.5                   18.2   
2                    6.4                    0.0   
3                    3.1                   11.3   
4                    0.0                    0.0   

   Resultado_Multiplicacao_EF2007  Resultado_Multiplicacao_EM2007  
0                         47659.4                             0.0  
1 

In [37]:
merged_df[['CO_MUNICIPIO', 'Nome do Município','Localização', 'Total_Urbana','Total_Rural', 'Total_Abandono_EF2007', 'Total_Abandono_EM2007', 'Resultado_Multiplicacao_EF2007','Resultado_Multiplicacao_EM2007']].head()

,CO_MUNICIPIO,Nome do Município,Localização,Total_Urbana,Total_Rural,Total_Abandono_EF2007,Total_Abandono_EM2007,Resultado_Multiplicacao_EF2007,Resultado_Multiplicacao_EM2007
0,1100015,ALTA FLORESTA D'OESTE,Rural,7687.0,2596.0,6.2,0.0,47659.4,0.0
1,1100015,ALTA FLORESTA D'OESTE,Urbana,7687.0,2596.0,5.5,18.2,42278.5,139903.4
2,1100023,ARIQUEMES,Rural,37912.0,3607.0,6.4,0.0,242636.8,0.0
3,1100023,ARIQUEMES,Urbana,37912.0,3607.0,3.1,11.3,117527.2,428405.6
4,1100031,CABIXI,Rural,2970.0,51.0,0.0,0.0,0.0,0.0
